In [ ]:

import pandas as pd
import os
import numpy as np

# 🚀 Ensure `xlsxwriter` is installed
try:
    import xlsxwriter
except ModuleNotFoundError:
    print("❌ ERROR: `xlsxwriter` is not installed. Please run `pip install xlsxwriter` and restart the script.")
    raise

"""
📊 **SNR Results Export to Excel**
This script **only** computes and saves SNR values into an Excel file.
"""

def save_snr_to_excel(images, probe, repo_directory):
    """
    Computes and saves SNR values into an Excel file, organized by probe type.

    🔹 **Parameters**:
    - `images` (dict): MRI image filenames as keys and image data as values.
    - `probe` (str): Selected probe (`p1` for 3FDG, `p2` for 3FDGal).
    - `repo_directory` (str): Path to the main project directory.

    📂 **Output**:
    - Saves `SNR_Results_P1.xlsx` (for 3FDG) or `SNR_Results_P2.xlsx` (for 3FDGal) in `snr_results/`
    """

    # 📂 Ensure results folder exists
    snr_results_dir = os.path.join(repo_directory, "snr_results")
    os.makedirs(snr_results_dir, exist_ok=True)

    # 🎯 Probe-Specific Mappings
    metabolite_map = {
        "p1": {"m1": "3FDGlucose", "m3": "3FDGluconic Acid", "m4": "3FDSorbitol"},
        "p2": {"m1": "3FDGalactose", "m3": "3FDGalactonic Acid", "m4": "3FDGalactitol"}
    }
    time_map = {"t1": "1hr", "t2": "2hr", "t3": "3hr"}

    # 🗂 Prepare Data Storage
    summary_data = []

    for file_path, img in images.items():
        snr, max_signal, noise, _ = compute_snr(np.array(img))

        # 🏷 Extract filename details
        parts = file_path.replace(".tif", "").split("_")
        if len(parts) < 4:
            continue  # Skip unexpected formats

        time_label = time_map.get(parts[2], parts[2])
        metabolite_label = metabolite_map[probe].get(parts[3], parts[3])

        # 📌 Store Data
        summary_data.append([file_path, metabolite_label, time_label, snr, max_signal, noise])

    # 📄 Create and Save Excel File
    output_excel = os.path.join(snr_results_dir, f"SNR_Results_{probe.upper()}.xlsx")
    df = pd.DataFrame(summary_data, columns=["Filename", "Metabolite", "Time", "SNR", "Max Signal", "Noise Std Dev"])
    
    with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
        df.to_excel(writer, sheet_name="SNR_Data", index=False)

    print(f"✅ SNR results saved successfully: {output_excel}")

# 📝 Define Paths
repo_directory = r"YOUR_LOCAL_PATH_HERE/FMRI-Metabolism-Quantification-main"

# 🎯 Set Probe Type ('p1' for 3FDG or 'p2' for 3FDGal)
probe = "p2"


# ✅ Ensure 'images' exists
if 'images' not in globals():
    raise NameError("❌ ERROR: 'images' variable is not defined! Ensure that load_and_display_mri.ipynb creates this variable.")

# 🔍 Compute & Save SNR Data (No extra output)
save_snr_to_excel(images, probe, repo_directory)
✅ SNR results saved successfully: C:\Users\libis\Desktop\FMRI-Metabolism-Quantification-main\snr_results\SNR_Results_P2.xlsx